In [ ]:
import torch
import torch.nn as nn
import fitz  # PyMuPDF
import os
from torchdiffeq import odeint_adjoint as odeint
from sentence_transformers import SentenceTransformer


# --- 1. CONTINUOUS-TIME REASONING (Neural ODE) ---
class ODEFunc(nn.Module):
    def __init__(self, dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, 128),
            nn.Tanh(),
            nn.Linear(128, dim)
        )

    def forward(self, t, h):
        return self.net(h)


# --- 2. HYBRID MODEL ---
class HybridNeuralODE(nn.Module):
    def __init__(self, hidden_dim):
        super(HybridNeuralODE, self).__init__()
        self.ode_func = ODEFunc(hidden_dim)

        # External memory bank (stores important facts)
        self.memory_bank = []

        self.surprise_threshold = 0.35
        self.decoder = nn.Linear(hidden_dim, 1)

    def forward(self, x_sequence):
        # Initial hidden state
        h = x_sequence[0].unsqueeze(0)

        integration_time = torch.linspace(0, 1, 2)

        for i in range(1, len(x_sequence)):
            # A. Continuous reasoning
            h = odeint(self.ode_func, h, integration_time)[-1]

            # B. Surprise detection
            surprise_score = torch.dist(h, x_sequence[i])

            if surprise_score > self.surprise_threshold:
                self.memory_bank.append(
                    x_sequence[i].detach().clone()
                )

            # C. Memory retrieval
            if len(self.memory_bank) > 0:
                memory_info = torch.mean(
                    torch.stack(self.memory_bank), dim=0
                )
                h = h + 0.05 * memory_info

        # Final decoding
        return self.decoder(h)


# --- 3. EXECUTION PIPELINE ---
def run_project(file_path):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return

    print("Loading encoder and extracting PDF text...")
    encoder = SentenceTransformer("all-MiniLM-L6-v2")

    doc = fitz.open(file_path)
    sentences = [page.get_text().replace("\n", " ") for page in doc]

    # Encode text
    input_vectors = encoder.encode(
        sentences, convert_to_tensor=True
    )

    # convert inference tensors to normal tensors
    input_vectors = input_vectors.clone().detach()

    model = HybridNeuralODE(hidden_dim=input_vectors.shape[1])

    print(f"Reasoning over {len(sentences)} document steps...")

    # No gradients needed (inference-only)
    with torch.no_grad():
        result = model(input_vectors)

    print("\n" + "=" * 40)
    print("HYBRID MODEL ANALYSIS COMPLETE")
    print(f"Final Understanding Score: {result.item():.4f}")
    print(f"Total Sentences Processed: {len(sentences)}")
    print(f"Significant Facts Stored in Memory: {len(model.memory_bank)}")
    print("=" * 40)


if __name__ == "__main__":
    FILE_NAME = "HarryPotter.pdf"  
    run_project(FILE_NAME)


c:\Users\Meera\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Loading encoder and extracting PDF text...
Reasoning over 250 document steps...

HYBRID MODEL ANALYSIS COMPLETE
Final Understanding Score: 80.3733
Total Sentences Processed: 250
Significant Facts Stored in Memory: 249
